# Unlockable Chest

create a chest that can be unlocked by a string:
- an either successful or unsuccessful unlocking attempt takes at least a few seconds, but does not eat up memory
- another string is retreived when unlocked (a random string if unsuccessful)

In [1]:
import hashlib
import heapq
from dataclasses import dataclass

In [2]:
DEFAULT_ITERS = 2_000_000
DEFAULT_LIMIT = 2000


@dataclass
class Chest:
    places: list
    iter_count: int = DEFAULT_ITERS
    hex_limit: int = DEFAULT_LIMIT

    def unlock(self, key):
        hexes = get_hexes(self.iter_count, self.hex_limit, key)
        out = ""
        for h_ind, sl, sr in self.places:
            out += chr(int(hexes[h_ind][sl:sr], base=16))
        return out

    @classmethod
    def create_chest(
        cls, key, secret, iter_count=DEFAULT_ITERS, hex_limit=DEFAULT_LIMIT
    ):
        secret_ord = [hex(ord(c))[2:] for c in secret]
        hexes = get_hexes(iter_count, hex_limit, key)
        places = []
        letter_ind = 0
        for h_ind, h in enumerate(hexes):
            try:
                letter_ord = secret_ord[letter_ind]
            except IndexError:
                break
            try:
                sub_ind = h.index(letter_ord)
            except ValueError:
                continue
            places.append((h_ind, sub_ind, sub_ind + len(letter_ord)))
            letter_ind += 1
        else:
            raise ValueError("Too few iterations")
        return cls(places, iter_count)


def get_hexes(iter_count, limit, key):
    hm5 = hashlib.md5(key.lower().encode("utf-8"))
    hexes = []
    for _ in range(iter_count):
        hm5.update(hm5.hexdigest().encode("utf-8"))
        new_hex = hm5.hexdigest()
        (heapq.heappush if len(hexes) < limit else heapq.heappushpop)(hexes, new_hex)
    return [h[-10:] for h in sorted(hexes)]

In [3]:
chest = Chest.create_chest("abcd", "Something Cool")

In [6]:
chest

Chest(places=[(44, 7, 9), (54, 2, 4), (63, 8, 10), (88, 5, 7), (107, 3, 5), (116, 3, 5), (143, 8, 10), (144, 7, 9), (194, 1, 3), (199, 0, 2), (209, 0, 2), (226, 1, 3), (280, 5, 7), (292, 5, 7)], iter_count=2000000, hex_limit=2000)

In [4]:
chest.unlock("XYZ")

'\re\xa0Ífo\x9e\x0b\x86Ô\x93\x06$Ý'

In [5]:
chest.unlock("abcd")

'Something Cool'